# Notebook describing an Issue when importing a multi-level DataFrame into Xarray
#### The problem appears when the multi-level is generated from the concatenation of two DataSeries. It cannot be reproduced for a multi-level DataFrame generated using the dataframe *groupby()* method

In [210]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import xarray as xr


#### List library version

In [211]:
import pkg_resources
# list packages to be checked
root_packages = [
    'pandas', 'numpy', 'xarray']
# print versions, but check if package is imported first
for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print(f"{m.project_name}=={m.version}")

xarray==0.15.1
pandas==1.0.3
numpy==1.18.1


<br><br>
## 1. Create DataFrame from 3D data
#### 1.1 Load Data
We are loading here a dummy version of the 3D velocity fields of ocean currents recorded in the North Atlantic along the Extended Ellet Line section (http://prj.noc.ac.uk/ExtendedEllettLine/project-information). 
Each row of the csv file corresponds to an element (i,j,k) of a 3D grid where the dimensions are: 
1. Horizontal distance (measurements are made between Iceland and Scotland every 15-30km),
2. Vertical depth levels (measurements are made from the sea surface to the bottom of the ocean ~3000m in this region). 
3. Time 

The total number of rows is the product of the 3 dimensions (Depth x Lon x Time). The number of columns of the CSV file is the sum of the number of dimensions (3 in the case of a 3D grid) with the numbers of variables extracted at each grid point (e.g. zonal velocity U, meridional velocity V)  + number of other informative variables (e.g. distance along the section, station number, cruise unique Identification number)"


In [212]:
pathdir = '../data/raw/csv_ctdgrid'
file4 = pathdir+'/'+'EEL_LADCP_3Dfield_dummy.csv'
dflad = pd.read_csv(file4,sep=',', index_col=None, 
                     header=0)
dflad=dflad.round({'LADCP_U': 3, 'LADCP_V': 3})
dflad

,CruiseID,Year,Staname,Stanumber,Depth,LADCP_U,LADCP_V
0,d22396,1996,13G,13,25,25913.96,-25913.96
1,d22396,1996,13G,13,35,35913.96,-35913.96
2,d22396,1996,13G,13,45,45913.96,-45913.96
3,d22396,1996,13G,13,55,55913.96,-55913.96
4,d22396,1996,13G,13,65,65913.96,-65913.96
...,...,...,...,...,...,...,...
76029,dy078,2017,IB23S,69,65,65969.17,-65969.17
76030,dy078,2017,IB23S,69,75,75969.17,-75969.17
76031,dy078,2017,IB23S,69,85,85969.17,-85969.17
76032,dy078,2017,IB23S,69,95,95969.17,-95969.17


<br><br>
#### Load 2nd dataframe containing metadata on datasets dimensions.

In [213]:
pathdir = '../data/raw/csv_ctdgrid'
file3 = pathdir+'/'+'EELCTDandLADCP_refpos_withstanumber.csv'
dfloc = pd.read_csv(file3,sep=',', index_col=None, 
                     header=0)
# Make sure the station name are sorted by their distance along the section
sdfloc = dfloc.sort_values('Refdist', ascending=True)
sdfloc['Refdist']=sdfloc['Refdist'].apply(int)
sdfloc

,Staname,Stanumber,Refdist,LonSta,LatSta,DepthSta
68,IB23S,69,0,-20.215,63.317,120
67,IB22S,68,13,-20.067,63.216,670
66,IB21S,67,25,-19.916,63.133,1030
65,IB20S,66,55,-19.551,62.917,1400
64,IB19S,65,84,-19.668,62.667,1670
...,...,...,...,...,...,...
4,5G,5,1277,-6.600,56.733,80
3,4G,4,1286,-6.450,56.733,115
2,3G,3,1292,-6.367,56.708,70
1,2G,2,1298,-6.283,56.683,30


\
To help track down the problem of conversion into a Dataset, the variables LADCP_U and LADCP_V have been replaced by a code related to their dimensions. 

Each element of LADCP_U is unique and defined as:
*Depth*1000 + 900 + *Stanumber* + *Year*(format YY)/100

LADCP_V is equal to - LADCP_U

For example, the zonal velocity U measured at 35m on Station 13G (station 13) in 1996 has a value of 35913.96. Or the meridional velocity V measured at 95m at Station IB23S (station number 69) in 2017 has a vale of -95969.17

<br><br>


#### 1.2 Create Multi-level DataFrame from concatenation of DataSeries

In [214]:
# Calculate depth average current for each year at each station using a pivot table
df_U = dflad.pivot_table(values='LADCP_U', index="Year", columns="Staname").round(3)
df_V = dflad.pivot_table(values='LADCP_V', index="Year", columns="Staname").round(3)
df_V.head()

Staname,10G,13G,14G,15G,8G,9G,A,B,C,D,...,L,M,N,O,P,Q,Q1,R,S,T
Year,,,,,,,,,,,,,,,,,,,,,
1996,NaN,-50913.96,-50914.96,-50916.96,NaN,NaN,-55936.96,-75935.96,-140934.96,-555933.96,...,-1075925.96,-1090924.96,-1040923.96,-955922.96,-715921.96,-130919.96,-135920.96,-50918.96,-50917.96,-55915.96
1997,NaN,NaN,NaN,NaN,NaN,NaN,-80936.97,NaN,NaN,-520933.97,...,NaN,-1110924.97,-990923.97,-985922.97,-725921.97,-150919.97,-150920.97,-60918.97,NaN,-85915.97
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-550933.98,...,NaN,-1130924.98,-990923.98,-990922.98,-720921.98,-160919.98,-160920.98,-70918.98,NaN,NaN
1999,NaN,NaN,NaN,NaN,NaN,NaN,-75936.99,-85935.99,-135934.99,-520933.99,...,-1075925.99,-1085924.99,-1035923.99,-675922.99,-675921.99,-165919.99,-380920.99,NaN,NaN,NaN
2000,-100910.0,NaN,NaN,-65916.00,NaN,-70909.0,NaN,NaN,NaN,NaN,...,-1060925.00,-1115924.00,-1055923.00,-955922.00,-725921.00,-180919.00,-355920.00,-60918.00,-65917.00,-65915.00


<br><br>
Sort the dataframe rows according to another Dataframe of "reference" where the location of the stations has been sorted according to their distance along the survey line (the *variable *Refdist*). We use the reference dataframe sdfloc. The function also merge additional variables contained in the DataFrame of Reference sdfloc (e.g. lat, lon, depth of the station).

In [215]:
def sortEELPV(dftosort, refdf=sdfloc):
    """ 
    Function to sort the row of a dataframe according to another dataframe. Here the rows are sorted according to another 
    dataframe containing the reference distance along the Extended Ellet Line section. 
    Additional metadata corresponding to the reference station (e.g. lat, lon, depth) are also merged into the new dataframe.

    """
    dfnew=pd.merge(refdf,
                   dftosort,
                   how='left',
                   on='Staname').round(3)
    dfnew = dfnew.set_index('Staname',drop=True) 
    return dfnew


# Sort the dataframe row according to location of the station along the section.
# We use the reference dataframe sdfloc and merge additional metadata from sdfloc (lat, lon) into the new dataframe
df_U = sortEELPV(df_U.T, refdf=sdfloc)
df_V = sortEELPV(df_V.T, refdf=sdfloc)


\
Format DataSeries and concatenate them into a Multilevel DataFrame

In [216]:
df_V = df_V.drop(columns=['Refdist', 'Stanumber','LonSta', 'LatSta', 'DepthSta'])
df_Vs=df_V.stack().rename_axis(['Staname', 'Year']).rename('V')

df_U = df_U.drop(columns=['Refdist', 'Stanumber', 'LonSta', 'LatSta', 'DepthSta'])
df_Us=df_U.stack().rename_axis(['Staname', 'Year']).rename('U')

dfs=pd.concat([df_Us,df_Vs],axis=1)
dfs1

U         V
Staname Year                    
IB23S   2005  65969.05 -65969.05
        2006  65969.06 -65969.06
        2010  60969.10 -60969.10
        2011  60969.11 -60969.11
        2014  60969.14 -60969.14
...                ...       ...
9G      2017  75909.17 -75909.17
8G      2005  85908.05 -85908.05
        2015  85908.15 -85908.15
        2016  85908.16 -85908.16
        2017  85908.17 -85908.17

[621 rows x 2 columns]

<br><br>
#### 1.3 Create Multi-level DataFrame using *groupby*.....

In [217]:
df = dflad.groupby(by=['Staname','Stanumber','Year']).mean().drop(columns='Depth').round(3).rename(columns={"LADCP_U": "U", "LADCP_V": "V"})
df

U          V
Staname Stanumber Year                      
10G     10        2000  100910.00 -100910.00
                  2010  105910.10 -105910.10
                  2015  105910.15 -105910.15
                  2016  105910.16 -105910.16
                  2017  105910.17 -105910.17
...                           ...        ...
T       15        2006   65915.06  -65915.06
                  2009   65915.09  -65915.09
                  2010   65915.10  -65915.10
                  2015   65915.15  -65915.15
                  2016   65915.16  -65915.16

[621 rows x 2 columns]

\
By default the multilevel Dataframe start at Station 10G and finish at station T (Panda sorts the rows/index in ascending order). 

To get the list of the Station name used as index in this Dataframe we can use the *.index* and *.get_level_values* methods:


In [218]:
df.index.get_level_values('Staname').unique()

Index(['10G', '13G', '14G', '15G', '8G', '9G', 'A', 'B', 'C', 'D', 'E', 'F',
       'G', 'H', 'I', 'IB1', 'IB10', 'IB11', 'IB11A', 'IB12', 'IB12A', 'IB13',
       'IB13A', 'IB14', 'IB15', 'IB16', 'IB16A', 'IB17', 'IB18S', 'IB19S',
       'IB1A', 'IB2', 'IB20S', 'IB21S', 'IB22S', 'IB23S', 'IB2A', 'IB3',
       'IB3A', 'IB4', 'IB4A', 'IB4B', 'IB4C', 'IB5', 'IB6', 'IB7', 'IB8',
       'IB9', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'Q1', 'R', 'S', 'T'],
      dtype='object', name='Staname')

\
We can clearly see that it is different from the sequential order of the stations given by the *Stanumber* variable

In [219]:
df.index.get_level_values('Stanumber').unique()

Int64Index([10, 13, 14, 16,  8,  9, 36, 35, 34, 33, 32, 31, 30, 29, 28, 37, 52,
            53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 38, 39, 66, 67,
            68, 69, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 27, 26, 25,
            24, 23, 22, 21, 19, 20, 18, 17, 15],
           dtype='int64', name='Stanumber')

\
We created a second version of the dataframe sdf, where the index Staname is sorted according to the sequential number of the station from the highest number to the lowest (from Iceland to UK)

In [220]:
dfs2=df.reset_index().sort_values(['Stanumber','Year'],ascending=[False,True]).set_index(['Staname','Stanumber','Year'])
dfs2=dfs2.reset_index('Stanumber',drop=True)

dfs2

U         V
Staname Year                    
IB23S   2005  65969.05 -65969.05
        2006  65969.06 -65969.06
        2010  60969.10 -60969.10
        2011  60969.11 -60969.11
        2014  60969.14 -60969.14
...                ...       ...
9G      2017  75909.17 -75909.17
8G      2005  85908.05 -85908.05
        2015  85908.15 -85908.15
        2016  85908.16 -85908.16
        2017  85908.17 -85908.17

[621 rows x 2 columns]

<br><br>
## 2. Convert 2D DataFrames into DataArray

#### 2.1 From Multilevel Dataframe created from concatenation of DataSeries

In [221]:
dfs1

U         V
Staname Year                    
IB23S   2005  65969.05 -65969.05
        2006  65969.06 -65969.06
        2010  60969.10 -60969.10
        2011  60969.11 -60969.11
        2014  60969.14 -60969.14
...                ...       ...
9G      2017  75909.17 -75909.17
8G      2005  85908.05 -85908.05
        2015  85908.15 -85908.15
        2016  85908.16 -85908.16
        2017  85908.17 -85908.17

[621 rows x 2 columns]

\
Converstion into Dataset:

In [222]:
da1 = dfs1.to_xarray()
da1

<xarray.Dataset>
Dimensions:  (Staname: 60, Year: 15)
Coordinates:
  * Staname  (Staname) object 'IB23S' 'IB22S' 'IB21S' ... '10G' '9G' '8G'
  * Year     (Year) object 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017
Data variables:
    U        (Staname, Year) float64 nan nan nan nan ... 6.592e+04 6.592e+04 nan
    V        (Staname, Year) float64 nan nan nan ... -6.592e+04 -6.592e+04 nan

\
**The DataArray appear to be wrong** : the data indexed as station IB23, appear to be the one from station 10G!

Data corresponding to Station IB23 are:

In [239]:
print(f"V data corresponding to index 0:\n\n{da1.V[0,:]}\n")
print(f"%%%%%%%%%%%%%%%%%%%%\n")
print(f"Similar results when indexing using loc['IB23S',:]:\n\n{da1.V.loc['IB23S',:]}\n")

V data corresponding to index 0:

<xarray.DataArray 'V' (Year: 15)>
array([       nan,        nan,        nan,        nan, -100910.  ,
              nan,        nan,        nan, -105910.1 ,        nan,
              nan,        nan, -105910.15, -105910.16, -105910.17])
Coordinates:
    Staname  <U5 'IB23S'
  * Year     (Year) object 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017

%%%%%%%%%%%%%%%%%%%%

Similar results when indexing using loc['IB23S',:]:

<xarray.DataArray 'V' (Year: 15)>
array([       nan,        nan,        nan,        nan, -100910.  ,
              nan,        nan,        nan, -105910.1 ,        nan,
              nan,        nan, -105910.15, -105910.16, -105910.17])
Coordinates:
    Staname  <U5 'IB23S'
  * Year     (Year) object 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017



\
Which is similar to the original data correpsonding to Station 10G:

In [240]:
dfs1.V.loc['10G',:]

Staname  Year
10G      2000   -100910.00
         2010   -105910.10
         2015   -105910.15
         2016   -105910.16
         2017   -105910.17
Name: V, dtype: float64

\
Interestingly it doesnt seem to be the case if the DataFrame converted into DataArray is generated from the *.groubpy()* method (see next section)

<br><br>
#### 2.2 From Multilevel Dataframe created using *groupby*

Dataframe created by *groupby* seems perfectly identical to the DataFrame dfs1:

In [226]:
dfs2

U         V
Staname Year                    
IB23S   2005  65969.05 -65969.05
        2006  65969.06 -65969.06
        2010  60969.10 -60969.10
        2011  60969.11 -60969.11
        2014  60969.14 -60969.14
...                ...       ...
9G      2017  75909.17 -75909.17
8G      2005  85908.05 -85908.05
        2015  85908.15 -85908.15
        2016  85908.16 -85908.16
        2017  85908.17 -85908.17

[621 rows x 2 columns]

In [227]:
da2 = dfs2.to_xarray()
da2

<xarray.Dataset>
Dimensions:  (Staname: 60, Year: 15)
Coordinates:
  * Staname  (Staname) object '10G' '13G' '14G' '15G' '8G' ... 'Q1' 'R' 'S' 'T'
  * Year     (Year) int64 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017
Data variables:
    U        (Staname, Year) float64 nan nan nan nan ... 6.592e+04 6.592e+04 nan
    V        (Staname, Year) float64 nan nan nan ... -6.592e+04 -6.592e+04 nan

We can notice form the DataSet description that the Coordinate Staname is not sorted as in the original dataframe dfs2. The conversion to Xarray seems to have sorted the coordinate Staname by ascending order. Interestingly it was not the case for the creation of the DataSet da1.

In [228]:
print(da2.V[0,:])

<xarray.DataArray 'V' (Year: 15)>
array([       nan,        nan,        nan,        nan, -100910.  ,
              nan,        nan,        nan, -105910.1 ,        nan,
              nan,        nan, -105910.15, -105910.16, -105910.17])
Coordinates:
    Staname  <U3 '10G'
  * Year     (Year) int64 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017


\
In this case, the data for IB23S are similar that in the original Dataframe

In [229]:
print(da2.V.loc['IB23S',:])

<xarray.DataArray 'V' (Year: 15)>
array([      nan,       nan,       nan,       nan,       nan, -65969.05,
       -65969.06,       nan, -60969.1 , -60969.11,       nan, -60969.14,
       -60969.15, -60969.16, -55969.17])
Coordinates:
    Staname  <U5 'IB23S'
  * Year     (Year) int64 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017


<br><br>
#### 2.3 More diagnostics
The Data corresponding to the last element of the coordinate Staname (8G), appear to be in reality the data from the coordinate T .... 

In [241]:
print(da1.V[-1,:])

<xarray.DataArray 'V' (Year: 15)>
array([-55915.96, -85915.97,       nan,       nan, -65915.  , -65915.05,
       -65915.06, -65915.09, -65915.1 ,       nan,       nan,       nan,
       -65915.15, -65915.16,       nan])
Coordinates:
    Staname  <U2 '8G'
  * Year     (Year) object 1996 1997 1998 1999 2000 ... 2013 2014 2015 2016 2017


In [243]:
dfs1.V.loc['T',:]

Staname  Year
T        1996   -55915.96
         1997   -85915.97
         2000   -65915.00
         2005   -65915.05
         2006   -65915.06
         2009   -65915.09
         2010   -65915.10
         2015   -65915.15
         2016   -65915.16
Name: V, dtype: float64

<br><br>
## 3. Conclusion

Both original DataFrame dfs1 and dfs2 appear completely similar:

In [178]:
dfs2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 621 entries, ('IB23S', 2005) to ('8G', 2017)
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   U       621 non-null    float64
 1   V       621 non-null    float64
dtypes: float64(2)
memory usage: 11.7+ KB


In [179]:
dfs1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 621 entries, ('IB23S', 2005) to ('8G', 2017)
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   U       621 non-null    float64
 1   V       621 non-null    float64
dtypes: float64(2)
memory usage: 14.2+ KB


\
However, for a reason unknown, the DataSet created from the conversion of the DataFrame *dfs1* is wrong. The problem appear to be in the DataSet coordinate Staname which has bot been sorted by ascending order while the Data Variable appear to have been. 

The original multi-level DataFrame has been generated by the concatenation of two DataSeries. 

Interestingly, this problem doesnt occur if the original Multi-level DataFrame is generated using the *grouby()* method...